In [2]:
import pandas as pd
import numpy as np
import os
import regex as re
from collections import Counter, defaultdict
import sys

CONST_A = 0
CONST_C = 1
CONST_G = 2
CONST_T = 3

CONST_NT_MAP = ['A', 'C', 'G', 'T']

def remove_duplicates_round(df,hamm_thres=4,merge_counts=False):
    seqs = list(df.Seq.values)
    counts = list(df.Counts.values)
    c = 0
    while c<(len(counts)-1):
        if(distance(seqs[c],seqs[c+1]))<hamm_thres:
            if(counts[c]>counts[c+1]):
                if(merge_counts):
                    counts[c]+=counts[c+1]
                del counts[c+1],seqs[c+1]
            else:
                if(merge_counts):
                    counts[c+1]+=counts[c]
                del counts[c],seqs[c]
        else:
            c+=1
    return pd.DataFrame({'Seq':seqs,'Counts':counts})

def remove_all_duplicates(sequences,counts,hamming_thresh=4,merge_counts=False):
    df = pd.DataFrame({'Seq':sequences,'Counts':counts})
    seq_len = len(sequences[0])
    
    print('Removing hamming neighbors on dimension:')
    
    for i in range(seq_len):
        df = df.ix[(df.Seq.str.slice(seq_len-i)+df.Seq.str.slice(i)).sort_values().index]
        df = remove_duplicates_round(df,hamm_thres=hamming_thresh,merge_counts=merge_counts)
        print(i)
    return df

def distance(astring, bstring) :
    distance = 0
    
    limit = len(astring)
    diff = len(bstring) - len(astring)
    if len(bstring) < len(astring) :
        limit = len(bstring)
        diff = len(astring) - len(bstring)
    
    for i in range(limit) :
        if astring[i] != bstring[i] :
            distance += 1
    return distance + diff

def increment_bp_map(seq, bp_map, magnitude=1) :
    for i in range(0, len(seq)) :
        if seq[i] == 'A' :
            bp_map[i][CONST_A] += magnitude
        elif seq[i] == 'C' :
            bp_map[i][CONST_C] += magnitude
        elif seq[i] == 'G' :
            bp_map[i][CONST_G] += magnitude
        elif seq[i] == 'T' :
            bp_map[i][CONST_T] += magnitude
    return bp_map

def get_consensus_sequence(bp_map) :
    seq = ''
    for i in range(0, len(bp_map)) :
        max_count = 0
        max_j = 0
        for j in range(0, 4) :
            if bp_map[i][j] > max_count :
                max_count = bp_map[i][j]
                max_j = j
        seq += CONST_NT_MAP[max_j]
    return seq

def get_hamming_neighbor_1(seq, seq_map, start_r, end_r) :
    for i in range(start_r, end_r) :
        for base1 in CONST_NT_MAP :
            mut_seq = seq[:i] + base1 + seq[i+1:]
            if mut_seq in seq_map :
                return mut_seq
    return None

def get_hamming_neighbor_2(seq, seq_map, start_r, end_r) :
    for i in range(start_r, end_r) :
        for j in range(i + 1, end_r) :
            for base1 in CONST_NT_MAP :
                for base2 in CONST_NT_MAP :
                    mut_seq = seq[:i] + base1 + seq[i+1:j] + base2 + seq[j+1:]
                    if mut_seq in seq_map :
                        return mut_seq
    return None


In [3]:
#a = [[0] * 4] * 6

a = []
for i in range(0, 6) :
    a.append([])
    for j in range(0, 4) :
        a[i].append(0)

at_seq = 'ATATAT'

a = increment_bp_map(at_seq, a)

print(a)

[[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1]]


In [4]:
r1 = 'C:/Users/Johannes/Desktop/apa_analysis/r1_apa_fr.fq'
r2 = 'C:/Users/Johannes/Desktop/apa_analysis/r2_apa_fr.fq'

In [5]:
dna_regex = re.compile(r"(CAGACACAGCTC){s<=1}")                 # Sequence found only in plasmid amplicon on read 2
upstream_regex = re.compile(r"(CAATTCTGCT[ACGTN]{40}CTAAAATATA){s<=2}") # 12x2 bp sequences flanking upstream randomized region
downstream_regex = re.compile(r"(AGTATGAAAC[ACGTN]{20}ACCCTTATCC){s<=2}") # 12x2 bp sequences flanking dnstream randomized region
seq_regex = re.compile(r"(CAATTCTGCT[ACGTN]{40}CTAAAATATA){s<=2}.*(AGTATGAAAC[ACGTN]{20}ACCCTTATCC){s<=2}")

is_dna = 'CAGACACAGCTC'

n10_a = re.compile(r"(TGTTAAGAAC[ACGTN]{10}CTGGTAACTGACCTTCAAAG){s<=3}")
n10_b = re.compile(r"(TGTTAAGAACAAGTT[ACGTN]{10}AACTGACCTTCAAAG){s<=3}")
n10_c = re.compile(r"(TGTTAAGAACAAGTTTGGCT[ACGTN]{10}ACCTTCAAAG){s<=3}")
n20_a = re.compile(r"([ACGTN]{20}CTGGTAACTGACCTTCAAAG){s<=3}")
n20_b = re.compile(r"(TGTTAAGAAC[ACGTN]{20}ACCTTCAAAG){s<=3}")
n20_c = re.compile(r"(TGTTAAGAACAAGTTTGGCT[ACGTN]{20}){s<=3}")

wt20_down = re.compile(r"(GATGTCTCGTGATCTGGTGT){s<=3}")


In [9]:
f = {}
f[0] = open(r1,'r')
f[1] = open(r2,'r')

head, seq, pr, q = ({} for i in range(4))
n_seqs, n_qual, xyseqs = ({} for i in range(3))
count = 0

dna_test_limit = 1
dna_count_map = {}
dna_quality_map = {}
dna_seq_map = {}
dna_lib_map = {}

valid_dna_seq_length = 113

valid_seq_count = 0

n20_a_wt_down_count = 0
n20_a_n20_down_count = 0
n20_b_wt_down_count = 0
n20_b_n20_down_count = 0
n20_c_wt_down_count = 0
n20_c_n20_down_count = 0

dna_bp_map = {}

dna_upstream_map = {}
dna_downstream_map = {}
dna_pas_map = {}
dna_seq_map = {}

matched_on_dict_count = 0
matched_on_hamming1_count = 0
matched_on_hamming2_count = 0

while True:
    for i in range(2):
        head[i] = f[i].readline()[:-1]
        seq[i] = f[i].readline()[:-1]
        pr[i] = f[i].readline()[:-1]
        q[i] = f[i].readline()[:-1]
    if len(q[1]) == 0:
        break # End of File
        
    dna_test = seq[1][12:24]
    dna_test_distance = distance(dna_test, is_dna)
    
    if dna_test_distance <= dna_test_limit :
    
        upstream_flank = re.search(upstream_regex, seq[0][12:200])
        downstream_flank = re.search(downstream_regex, seq[0][100:220])
        both_flank = re.search(seq_regex, seq[0][12:220])
        
        if upstream_flank != None and downstream_flank != None and both_flank != None:
            valid_seq_count += 1
            
            upstream_flank_seq = upstream_flank.group()
            downstream_flank_seq = downstream_flank.group()
            both_flank_seq = both_flank.group()
            
            n10_match = False
            if re.search(n10_a, upstream_flank_seq) or re.search(n10_b, upstream_flank_seq) or re.search(n10_c, upstream_flank_seq) :
                n10_match = True
            
            n20_match = False
            if re.search(n20_a, upstream_flank_seq) or re.search(n20_b, upstream_flank_seq) or re.search(n20_c, upstream_flank_seq) :
                n20_match = True
            
            if n10_match == False and n20_match == True and len(both_flank_seq) == valid_dna_seq_length :
                
                is_n20_down = True
                if re.search(wt20_down, downstream_flank_seq) :
                    is_n20_down = False
                
                upstream_seq = upstream_flank_seq[10:50]
                downstream_seq = downstream_flank_seq[10:30]
                pas_seq = both_flank_seq[50:65]
                full_seq = both_flank_seq[10:103]
                dna_lib = -1
                
                start_r = 0
                end_r = 40
                if re.search(n20_a, upstream_flank_seq) and is_n20_down == False :
                    n20_a_wt_down_count += 1
                    upstream_seq = upstream_seq[0:20] + 'CTGGTAACTGACCTTCAAAG'
                    dna_lib = 0
                    start_r = 0
                    end_r = 20
                elif re.search(n20_a, upstream_flank_seq) and is_n20_down == True :
                    n20_a_n20_down_count += 1
                    upstream_seq = upstream_seq[0:20] + 'CTGGTAACTGACCTTCAAAG'
                    dna_lib = 3
                    start_r = 0
                    end_r = 20
                elif re.search(n20_b, upstream_flank_seq) and is_n20_down == False :
                    n20_b_wt_down_count += 1
                    upstream_seq = 'TGTTAAGAAC' + upstream_seq[10:30] + 'ACCTTCAAAG'
                    dna_lib = 1
                    start_r = 10
                    end_r = 30
                elif re.search(n20_b, upstream_flank_seq) and is_n20_down == True :
                    n20_b_n20_down_count += 1
                    upstream_seq = 'TGTTAAGAAC' + upstream_seq[10:30] + 'ACCTTCAAAG'
                    dna_lib = 4
                    start_r = 10
                    end_r = 30
                elif re.search(n20_c, upstream_flank_seq) and is_n20_down == False :
                    n20_c_wt_down_count += 1
                    upstream_seq = 'TGTTAAGAACAAGTTTGGCT' + upstream_seq[20:40]
                    dna_lib = 2
                    start_r = 20
                    end_r = 40
                elif re.search(n20_c, upstream_flank_seq) and is_n20_down == True :
                    n20_c_n20_down_count += 1
                    upstream_seq = 'TGTTAAGAACAAGTTTGGCT' + upstream_seq[20:40]
                    dna_lib = 5
                    start_r = 20
                    end_r = 40
                
                
                dx, xd = upstream_flank.start(), upstream_flank.end()
                qualityscore = np.array([ord(i) - 33 for i in q[0][dx:xd]])

                new_member = True
                upstream_seq_key = upstream_seq
                if upstream_seq in dna_count_map :
                    new_member = False
                    matched_on_dict_count += 1
                else :
                    upstream_seq_h1 = get_hamming_neighbor_1(upstream_seq, dna_count_map, start_r, end_r)
                    if upstream_seq_h1 != None :
                        new_member = False
                        upstream_seq_key = upstream_seq_h1
                        matched_on_hamming1_count += 1
                    else :
                        upstream_seq_h2 = get_hamming_neighbor_2(upstream_seq, dna_count_map, start_r, end_r)
                        if upstream_seq_h2 != None :
                            new_member = False
                            upstream_seq_key = upstream_seq_h2
                            matched_on_hamming2_count += 1
                
                if new_member == True :
                    dna_count_map[upstream_seq_key] = 0
                    dna_quality_map[upstream_seq_key] = 0
                    dna_lib_map[upstream_seq_key] = dna_lib
                    dna_upstream_map[upstream_seq_key] = {}
                    dna_downstream_map[upstream_seq_key] = {}
                    dna_pas_map[upstream_seq_key] = {}
                    dna_seq_map[upstream_seq_key] = {}
                    
                dna_count_map[upstream_seq_key] += 1
                dna_quality_map[upstream_seq_key] += qualityscore

                if upstream_seq not in dna_upstream_map[upstream_seq_key] :
                    dna_upstream_map[upstream_seq_key][upstream_seq] = 1
                else :
                    dna_upstream_map[upstream_seq_key][upstream_seq] += 1
                
                if pas_seq not in dna_pas_map[upstream_seq_key] :
                    dna_pas_map[upstream_seq_key][pas_seq] = 1
                else :
                    dna_pas_map[upstream_seq_key][pas_seq] += 1
                
                if downstream_seq not in dna_downstream_map[upstream_seq_key] :
                    dna_downstream_map[upstream_seq_key][downstream_seq] = 1
                else :
                    dna_downstream_map[upstream_seq_key][downstream_seq] += 1
                
                if full_seq not in dna_seq_map[upstream_seq_key] :
                    dna_seq_map[upstream_seq_key][full_seq] = 1
                else :
                    dna_seq_map[upstream_seq_key][full_seq] += 1
                
                #if upstream_flank_seq not in dna_seq_map :
                #    dna_seq_map[upstream_flank_seq] = both_flank_seq
    if (count % 100000) == 0:
        print(count)
        print(str(valid_seq_count) + ' valid DNA reads extracted.')
        print('N20 A WT DOWN: ' + str(n20_a_wt_down_count))
        print('N20 A N20 DOWN: ' + str(n20_a_n20_down_count))
        print('N20 B WT DOWN: ' + str(n20_b_wt_down_count))
        print('N20 B N20 DOWN: ' + str(n20_b_n20_down_count))
        print('N20 C WT DOWN: ' + str(n20_c_wt_down_count))
        print('N20 C N20 DOWN: ' + str(n20_c_n20_down_count))
        
        print('Number of unique DNA members: ' + str(len(dna_count_map)))
        
        print('Matched on dictionary: ' + str(matched_on_dict_count))
        print('Matched on hamming 1: ' + str(matched_on_hamming1_count))
        print('Matched on hamming 2: ' + str(matched_on_hamming2_count))
    count += 1

print('COMPLETE')
print(str(valid_seq_count) + ' valid DNA reads extracted.')
print('N20 A WT DOWN: ' + str(n20_a_wt_down_count))
print('N20 A N20 DOWN: ' + str(n20_a_n20_down_count))
print('N20 B WT DOWN: ' + str(n20_b_wt_down_count))
print('N20 B N20 DOWN: ' + str(n20_b_n20_down_count))
print('N20 C WT DOWN: ' + str(n20_c_wt_down_count))
print('N20 C N20 DOWN: ' + str(n20_c_n20_down_count))

print('Number of unique DNA members: ' + str(len(dna_count_map)))

print('Matched on dictionary: ' + str(matched_on_dict_count))
print('Matched on hamming 1: ' + str(matched_on_hamming1_count))
print('Matched on hamming 2: ' + str(matched_on_hamming2_count))

f[0].close()
f[1].close()

0
1 valid DNA reads extracted.
N20 A WT DOWN: 0
N20 A N20 DOWN: 0
N20 B WT DOWN: 0
N20 B N20 DOWN: 0
N20 C WT DOWN: 0
N20 C N20 DOWN: 0
Number of unique DNA members: 0
Matched on dictionary: 0
Matched on hamming 1: 0
Matched on hamming 2: 0
100000
62334 valid DNA reads extracted.
N20 A WT DOWN: 6667
N20 A N20 DOWN: 5180
N20 B WT DOWN: 4112
N20 B N20 DOWN: 3718
N20 C WT DOWN: 5289
N20 C N20 DOWN: 5775
Number of unique DNA members: 30104
Matched on dictionary: 599
Matched on hamming 1: 29
Matched on hamming 2: 9
200000
125079 valid DNA reads extracted.
N20 A WT DOWN: 13322
N20 A N20 DOWN: 10495
N20 B WT DOWN: 8443
N20 B N20 DOWN: 7587
N20 C WT DOWN: 10561
N20 C N20 DOWN: 11406
Number of unique DNA members: 59284
Matched on dictionary: 2435
Matched on hamming 1: 76
Matched on hamming 2: 19
300000
187935 valid DNA reads extracted.
N20 A WT DOWN: 20027
N20 A N20 DOWN: 15951
N20 B WT DOWN: 12700
N20 B N20 DOWN: 11533
N20 C WT DOWN: 15827
N20 C N20 DOWN: 17156
Number of unique DNA members: 87

In [10]:
upstream_seq_map = {}
pas_seq_map = {}
downstream_seq_map = {}
seq_map = {}

for upstream_key in dna_upstream_map :
    upstream_bp_map = []
    pas_bp_map = []
    downstream_bp_map = []
    seq_bp_map = []
    
    for i in range(0, 40) :
        upstream_bp_map.append([])
        for j in range(0, 4) :
            upstream_bp_map[i].append(0)
    
    for i in range(0, 15) :
        pas_bp_map.append([])
        for j in range(0, 4) :
            pas_bp_map[i].append(0)
    
    for i in range(0, 20) :
        downstream_bp_map.append([])
        for j in range(0, 4) :
            downstream_bp_map[i].append(0)
    
    for i in range(0, 93) :
        seq_bp_map.append([])
        for j in range(0, 4) :
            seq_bp_map[i].append(0)
    
    upstream_list = list(dna_upstream_map[upstream_key].keys())
    pas_list = list(dna_pas_map[upstream_key].keys())
    downstream_list = list(dna_downstream_map[upstream_key].keys())
    seq_list = list(dna_seq_map[upstream_key].keys())
    
    if dna_count_map[upstream_key] > 2 :
        for upstream in dna_upstream_map[upstream_key] :
            upstream_bp_map = increment_bp_map(upstream, upstream_bp_map, magnitude=dna_upstream_map[upstream_key][upstream])
        upstream_seq_map[upstream_key] = get_consensus_sequence(upstream_bp_map)
    else :
        upstream_seq_map[upstream_key] = upstream_list[0]
    
    if dna_count_map[upstream_key] > 2 :
        for pas in dna_pas_map[upstream_key] :
            pas_bp_map = increment_bp_map(pas, pas_bp_map, magnitude=dna_pas_map[upstream_key][pas])
        pas_seq_map[upstream_key] = get_consensus_sequence(pas_bp_map)
    else :
        pas_seq_map[upstream_key] = pas_list[0]
    
    if dna_count_map[upstream_key] > 2 :
        for downstream in dna_downstream_map[upstream_key] :
            downstream_bp_map = increment_bp_map(downstream, downstream_bp_map, magnitude=dna_downstream_map[upstream_key][downstream])
        downstream_seq_map[upstream_key] = get_consensus_sequence(downstream_bp_map)
    else :
        downstream_seq_map[upstream_key] = downstream_list[0]
    
    if dna_count_map[upstream_key] > 2 :
        for seq in dna_seq_map[upstream_key] :
            seq_bp_map = increment_bp_map(seq, seq_bp_map, magnitude=dna_seq_map[upstream_key][seq])
        seq_map[upstream_key] = get_consensus_sequence(seq_bp_map)
    else :
        seq_map[upstream_key] = seq_list[0]


In [11]:
dna_upstream_list = []
dna_pas_list = []
dna_downstream_list = []
dna_seq_list = []
dna_upstream_count_list = []
dna_pas_count_list = []
dna_downstream_count_list = []
dna_seq_count_list = []
dna_count_list = []
dna_lib_list = []

for upstream in dna_count_map :
    dna_upstream_list.append(upstream_seq_map[upstream])
    dna_pas_list.append(pas_seq_map[upstream])
    dna_downstream_list.append(downstream_seq_map[upstream])
    dna_seq_list.append(seq_map[upstream])
    dna_upstream_count_list.append(len(dna_upstream_map[upstream]))
    dna_pas_count_list.append(len(dna_pas_map[upstream]))
    dna_downstream_count_list.append(len(dna_downstream_map[upstream]))
    dna_seq_count_list.append(len(dna_seq_map[upstream]))
    dna_count_list.append(dna_count_map[upstream])
    dna_lib_list.append(dna_lib_map[upstream])

df = pd.DataFrame({'upstream_seq':   dna_upstream_list,
                   'pas_seq':            dna_pas_list,
                   'downstream_seq':            dna_downstream_list,
                   'seq':            dna_seq_list,
                   'unique_upstream_seq_count': dna_upstream_count_list,
                   'unique_pas_seq_count': dna_pas_count_list,
                   'unique_downstream_seq_count': dna_downstream_count_list,
                   'unique_seq_count': dna_seq_count_list,
                   'library' :              dna_lib_list,
                   'read_count':     dna_count_list})

df = df.sort_values('read_count')

new_columns = ['upstream_seq', 'pas_seq', 'downstream_seq', 'seq', 'unique_upstream_seq_count', 'unique_downstream_seq_count', 'unique_pas_seq_count', 'unique_seq_count', 'library', 'read_count']
df.to_csv('apa_nextseq_v2_dna_20160922.csv', sep=',', header=True, columns=new_columns, index=False)

lib_summary = [0, 0, 0, 0, 0, 0]
for lib in dna_lib_list :
    lib_summary[lib] += 1

for i in range(0, len(lib_summary)) :
    print('Member count for library ' + str(i) + ': ' + str(lib_summary[i]))

Member count for library 0: 198739
Member count for library 1: 148707
Member count for library 2: 206215
Member count for library 3: 182715
Member count for library 4: 85797
Member count for library 5: 185209


In [12]:
dna_upstream_key_list = list(dna_count_map.keys())

print(len(dna_upstream_key_list))
print(len(dna_count_list))

hamming_thresh = 4

filtered_dna_df = remove_all_duplicates(dna_upstream_key_list, dna_count_list, hamming_thresh, merge_counts=False)

hamming_upstream_list = list(filtered_dna_df.Seq.values)
hamming_count_list = list(filtered_dna_df.Counts.values)

print(len(hamming_upstream_list))
print(len(hamming_count_list))
print('{:,}'.format(len(hamming_upstream_list)) + ' sequences with levenshtein d >= ' + str(hamming_thresh))


1007382
1007382
Removing hamming neighbors on dimension:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
1002130
1002130
1,002,130 sequences with levenshtein d >= 4


In [13]:
filtered_upstream_list = []
filtered_pas_list = []
filtered_downstream_list = []
filtered_seq_list = []
filtered_upstream_count_list = []
filtered_pas_count_list = []
filtered_downstream_count_list = []
filtered_seq_count_list = []
filtered_count_list = []
filtered_lib_list = []

for upstream in hamming_upstream_list :
    if dna_count_map[upstream] > 2 or (dna_count_map[upstream] == 2 and len(dna_pas_map[upstream]) == 1 and len(dna_downstream_map[upstream]) == 1) :
        filtered_upstream_list.append(upstream_seq_map[upstream])
        filtered_pas_list.append(pas_seq_map[upstream])
        filtered_downstream_list.append(downstream_seq_map[upstream])
        filtered_seq_list.append(seq_map[upstream])
        filtered_upstream_count_list.append(len(dna_upstream_map[upstream]))
        filtered_pas_count_list.append(len(dna_pas_map[upstream]))
        filtered_downstream_count_list.append(len(dna_downstream_map[upstream]))
        filtered_seq_count_list.append(len(dna_seq_map[upstream]))
        filtered_count_list.append(dna_count_map[upstream])
        filtered_lib_list.append(dna_lib_map[upstream])

print(len(filtered_upstream_list))

df = pd.DataFrame({'upstream_seq':   filtered_upstream_list,
                   'pas_seq':            filtered_pas_list,
                   'downstream_seq':            filtered_downstream_list,
                   'seq':            filtered_seq_list,
                   'unique_upstream_seq_count': filtered_upstream_count_list,
                   'unique_pas_seq_count': filtered_pas_count_list,
                   'unique_downstream_seq_count': filtered_downstream_count_list,
                   'unique_seq_count': filtered_seq_count_list,
                   'library' :              filtered_lib_list,
                   'read_count':     filtered_count_list})

df = df.sort_values('read_count')

new_columns = ['upstream_seq', 'pas_seq', 'downstream_seq', 'seq', 'unique_upstream_seq_count', 'unique_pas_seq_count', 'unique_downstream_seq_count', 'unique_seq_count', 'library', 'read_count']
df.to_csv('apa_nextseq_v2_dna_filtered_20160922.csv', sep=',', header=True, columns=new_columns, index=False)


lib_summary = [0, 0, 0, 0, 0, 0]
for lib in filtered_lib_list :
    lib_summary[lib] += 1

for i in range(0, len(lib_summary)) :
    print('Member count for library ' + str(i) + ': ' + str(lib_summary[i]))

691887
Member count for library 0: 146754
Member count for library 1: 99722
Member count for library 2: 128534
Member count for library 3: 122777
Member count for library 4: 68262
Member count for library 5: 125838
